In [38]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle


In [39]:
## load the trained model scaler pickel onehot 
model= load_model('model.h5')

# load the encode and scaller 

with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)  

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [ ]:
# sample input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
# # sample input data for prediction
# input_data = {
#     'CreditScore': 100,
#     'Geography': 'France',
#     'Gender': 'Male',
#     'Age': 30,
#     'Tenure': 3,
#     'Balance': 10000,
#     'NumOfProducts': 2,
#     'HasCrCard': 1,
#     'IsActiveMember': 1,
#     'EstimatedSalary': 10000
# }

In [42]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)
geo_encoded_df


c:\Users\aarav\Desktop\ANN CLASSIFICATION\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [43]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
input_df = pd.DataFrame([input_data])   
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,France,Male,30,3,10000,2,1,1,10000


In [45]:
##Encode categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,France,1,30,3,10000,2,1,1,10000


In [46]:
##cocatinae
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,100,1,30,3,10000,2,1,1,10000,1.0,0.0,0.0


In [47]:
## scalling the input data 
input_scaled = scaler.transform(input_df)
input_scaled

array([[-5.72485068,  0.91324755, -0.84593077, -0.69539349, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -1.57229902,  1.00150113,
        -0.57946723, -0.57638802]])

In [48]:
#prediction churn 
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 53ms/step


array([[0.01077664]], dtype=float32)

In [49]:
prediction_proba = prediction[0][0]

In [50]:
prediction_proba

0.010776645

In [51]:
if prediction_proba > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_proba:.2f}")  
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_proba:.2f}")    

The customer is unlikely to churn with a probability of 0.01
